This file contains all definitions needed for the scenario generator.

Please call the 'generate_scenario' function to get a new, random scenario. Note you can enter a custom random seed.

Please note that scenarios are generated using the following logic:
1. Under the 'Default values' markdown, you can see the expected number of passengers for evening rush hour, for any given workday.
2. This expected count across two hours is broken down into all of its five minute intervals (24 intervals total)
3. If E is the expected count for any line, we then have an expected E/24 count for each interval.
4. We could then generate a count for each inteval and line, so that it follows a Normal distribution with mean E/24 and std. dev 0.15*(E/24)
5. Since Normal could return negative, get max(count, 0), meaning nobody showed up to take the line (should happen rarely)
6. Then, we floor the returned value, as Normal can return non integer.

In [157]:
import numpy as np
from typing import Dict, Tuple
from time import time_ns

Default values

In [158]:
EXP_RIDERSHIP: Dict[str, int] = {
    'Lakeshore West': 33239,
    'Lakeshore East': 25769,
    'Kitchener': 15071,
    'Milton': 13138,
    'Barrie': 8636,
    'Stouffville': 7341,
    'Richmond Hill': 4642
}

SPREAD = 0.10 #Modify this to get scenarios further from the exp values above

Function for scenarios

In [159]:
def generate_scenario(seed: int | str = None) -> Dict[str, Dict[str, int]]:
    """
    Generates ridership demand scenario.

    Returns dict of all time interval and line combinations.

    seed: int | str => can be used for replication purposes. Random seed to use.
    """

    if seed is None: seed = time_ns()

    if type(seed) is not int: seed = sum([ord(char) for char in seed])

    np.random.seed(seed%(2**32-1))

    scenario: Dict[str, Dict[str, int]] = {}

    #Initializing dicts
    for line in EXP_RIDERSHIP.keys():
        scenario[line] = {}

    #Iterating lines
    for line, exp_demand in EXP_RIDERSHIP.items():
        mu = exp_demand/24
        sigma = mu*SPREAD

        #Iterating time intervals
        for t in range(0, 120, 5):
            demand = np.random.normal(loc=mu, scale=sigma)
            scenario[line][f"{4+int(t/60)}:{(t-60*int(t/60)):02}"] = max(0, int(demand))

    return scenario



In [160]:
generate_scenario("MIE562")

{'Lakeshore West': {'4:00': 1566,
  '4:05': 1326,
  '4:10': 1247,
  '4:15': 1549,
  '4:20': 1438,
  '4:25': 1595,
  '4:30': 1245,
  '4:35': 1564,
  '4:40': 1427,
  '4:45': 1476,
  '4:50': 1421,
  '4:55': 1394,
  '5:00': 1488,
  '5:05': 1285,
  '5:10': 1400,
  '5:15': 1196,
  '5:20': 1498,
  '5:25': 1392,
  '5:30': 1461,
  '5:35': 1394,
  '5:40': 1411,
  '5:45': 1714,
  '5:50': 1230,
  '5:55': 1490},
 'Lakeshore East': {'4:00': 1202,
  '4:05': 1099,
  '4:10': 1090,
  '4:15': 1291,
  '4:20': 913,
  '4:25': 1036,
  '4:30': 825,
  '4:35': 1179,
  '4:40': 994,
  '4:45': 1225,
  '4:50': 971,
  '4:55': 1208,
  '5:00': 900,
  '5:05': 1059,
  '5:10': 1109,
  '5:15': 1073,
  '5:20': 965,
  '5:25': 1102,
  '5:30': 1091,
  '5:35': 1051,
  '5:40': 1005,
  '5:45': 1234,
  '5:50': 909,
  '5:55': 938},
 'Kitchener': {'4:00': 647,
  '4:05': 614,
  '4:10': 523,
  '4:15': 646,
  '4:20': 622,
  '4:25': 612,
  '4:30': 605,
  '4:35': 741,
  '4:40': 697,
  '4:45': 674,
  '4:50': 719,
  '4:55': 583,
  '5:00':

In [161]:
generate_scenario()

{'Lakeshore West': {'4:00': 1265,
  '4:05': 1143,
  '4:10': 1297,
  '4:15': 1277,
  '4:20': 1455,
  '4:25': 1541,
  '4:30': 1392,
  '4:35': 1408,
  '4:40': 1405,
  '4:45': 1422,
  '4:50': 1604,
  '4:55': 1402,
  '5:00': 1346,
  '5:05': 1289,
  '5:10': 1388,
  '5:15': 1036,
  '5:20': 1352,
  '5:25': 1296,
  '5:30': 1290,
  '5:35': 1553,
  '5:40': 1240,
  '5:45': 1463,
  '5:50': 1498,
  '5:55': 1598},
 'Lakeshore East': {'4:00': 1172,
  '4:05': 1076,
  '4:10': 1213,
  '4:15': 1186,
  '4:20': 1036,
  '4:25': 1159,
  '4:30': 1013,
  '4:35': 1258,
  '4:40': 915,
  '4:45': 1098,
  '4:50': 1263,
  '4:55': 1017,
  '5:00': 1086,
  '5:05': 1143,
  '5:10': 963,
  '5:15': 1223,
  '5:20': 1159,
  '5:25': 933,
  '5:30': 982,
  '5:35': 891,
  '5:40': 1014,
  '5:45': 1035,
  '5:50': 999,
  '5:55': 1088},
 'Kitchener': {'4:00': 564,
  '4:05': 693,
  '4:10': 600,
  '4:15': 698,
  '4:20': 710,
  '4:25': 533,
  '4:30': 638,
  '4:35': 614,
  '4:40': 717,
  '4:45': 596,
  '4:50': 600,
  '4:55': 561,
  '5:00